# Practica 2 Bloque 2 - Mineria de textos.

## Ensemble de clasificación de textos cientificos.

## Instalación de paquetes e importación de las librerias necesarias

In [ ]:
!pip install tensorflow==2.15.0
!pip install transformers==4.32.1
!pip install scikit-learn==1.4.1.post1
!python -m spacy download es_core_news_sm

In [ ]:
import os

#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter

# para pre-procesamiento del texto y extraer características
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import EnglishStemmer

# algoritmos de clasificación
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# para construir pipelines
from sklearn.pipeline import Pipeline

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.multiclass import unique_labels

# para guardar el modelo
import pickle


# Librerias utilizadas por mi para diferentes tareas, como la definición del pipeline, la importación de diferentes clasificadores y la creación de gráficos.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.pipeline import FeatureUnion
from nltk.corpus import wordnet
import spacy
import es_core_news_sm
import matplotlib.pyplot as plt

# para guardar el modelo
import pickle
import tensorflow as tf
import keras

# para visualizar el modelo
from IPython.core.display import display

# algoritmos de clasificación, tokenizadores, etc.
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig, TFDistilBertMainLayer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.tree import plot_tree

print('Done!')

La siguiente función se ha creado para definir la semilla de aleatoriedad en algunos de los paquetes que se utilizan en la práctica, con el objetivo de obtener el mismo resultado en diferentes ejecuciones.

In [ ]:
def set_random_seed(seed_value):
    tf.random.set_seed(seed_value)
    spacy.util.fix_random_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
set_random_seed(199)

## Obtencion de los Datasets

El conjunto de datos escogido para la realizacion de la practica se encuentra en Kaggle en el siguiente enlace: https://www.kaggle.com/datasets/vivmankar/physics-vs-chemistry-vs-biology.

Se trata de una serie de textos cientificos junto a su categoria: textos sobre biologia, fisica y quimica.

Para ello he decidido juntar los datasets de entrenamiento y validacion de Kaggle para realizar mis propias divisiones, a continuación se lee el archivo .CSV que contiene los textos y las categorias de estos.

In [ ]:
data = pd.read_csv('/kaggle/input/datap2bloq2mintext/data.csv')

## Analisis exploratorio del conjunto de datos


In [ ]:
data

Como se observa, se trata de un dataset simple, con la columna objetivo y la columna que dispone de los textos.

In [ ]:
text_col = 'Comment'  # columna del dataframe que contiene el texto (depende del formato de los datos)
class_col = 'Topic'  # columna del dataframe que contiene la clase (depende del formato de los datos)

# obtener algunas estadísticas sobre los datos
categories = sorted(data[class_col].unique(), reverse=False)
hist= Counter(data[class_col]) 
print(f'Total de instancias -> {data.shape[0]}')
print('Distribución de clases:')
for item in sorted(hist.items(), key=lambda x: x[0]): print(f'    {item[0]}: {round(item[1]/len(data[class_col]), 3)}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {data[text_col][0]}')
print(f'Categoría del comentario -> {data[class_col][0]}')

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

print('Done!')

En el grafico anterior se muesstra la distribución de las clases, se puede observar a simple vista que hay muchos mas textos sobre biologia y quimica que sobre fisica, por lo que se procederá a un undersampling de las clases mayoritarias, para obtener un numero identico de instancias de cada clase.

## Preprocesamiento del Dataset

Para eliminar datos innecesarios, se procede a eliminar la columna 'id'.

In [ ]:
data.drop(columns='id', inplace=True)

data[class_col].value_counts()

En el resultado de la ejecución anterior se listan el total de instancias de cada clase.

En la siguiente celda de código se aplicará el $Undersampling$ a los textos sobre biologia y sobre quimica: para ello he escogido una muestra de tamaño $n=2650$ (para tener el mismo numero de instancias que los textos sobre fisica) del conjunto de datos, la he concatenado con los datos de fisica y finalmente he mezclado (shuffling).

NOTA: Como el entrenamiento de los modelos requiere de mucho tiempo de ejecución, se ha decidido emplear únicamente 500 muestras de cada categoria.

Finalmente se han guardado los datos preprocesados en una variable para poder utilizarlos posteriormente, sin tener que volver a realizar todo el preproceso de estos.

In [ ]:

''' #UNDERSAMPLING
# Tamaño de la muestra
n = 2650

# Muestra de tamaño n
muestra_bio = data[data[class_col] == 'Biology'].sample(n=n, random_state=1)
muestra_quim = data[data[class_col] == 'Chemistry'].sample(n=n, random_state=1)

# Combinar los datos
data = pd.concat([data[data[class_col] == 'Physics'], muestra_bio, muestra_quim])

# Mezclar los datos (shuffling)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Guardar los datos originales tras el preproceso para utilizarlos posteriormente
datos_procesados = data

# Imprimir total de cada clase
data[class_col].value_counts()

'''

#MUESTRAS REDUCIDAS
n = 500

# Muestra de tamaño n
muestra_bio = data[data[class_col] == 'Biology'].sample(n=n, random_state=1)
muestra_quim = data[data[class_col] == 'Chemistry'].sample(n=n, random_state=1)
muestra_fisica = data[data[class_col] == 'Physics'].sample(n=n, random_state=1)

# Combinar las muestras de cada clase
data_muestra = pd.concat([muestra_bio, muestra_quim, muestra_fisica])

# Mezclar los datos (shuffling)
data_muestra = data_muestra.sample(frac=1, random_state=42).reset_index(drop=True)

# Guardar los datos de muestra preprocesados para utilizarlos posteriormente
data = data_muestra

# Imprimir total de cada clase
data[class_col].value_counts()

In [ ]:
categories = sorted(data[class_col].unique(), reverse=False)
hist= Counter(data[class_col]) 
for item in sorted(hist.items(), key=lambda x: x[0]): print(f'    {item[0]}: {round(item[1]/len(data[class_col]), 3)}')
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()
print('Done!')

En este punto ya se dispone del conjunto de datos preparado para el entrenamiento de los diferentes modelos y la creación del ensemble.

## Conjunto de datos de entrenamiento y validacion

A continuación se divide el conjunto de datos en datos de entrenamiento y validación. En este caso he elegido realizar una división del 75% de los datos para el entrenamiento de los modelos y el 25% restante para su validación.

In [ ]:
seed = 0  # fijar random_state para reproducibilidad
train, val = train_test_split(data, test_size=.25, stratify=data[class_col], random_state=seed)

print('Done!')

# Primer Modelo - Pipeline simple de clasificación

El primero de los 3 modelos que se crearán para la clasificación de textos cientificos es el visto al inicio de la asignatura, un pipeline simple de clasificación textual.

### Funciones necesarias

A continuación se muestran las funciones necesarias para la predicción y la evaluación del modelo, asi como otras funciones auxiliares y un listado de stopwords que se eliminarán del texto.

In [ ]:
#listado de stopwords. Este listado también se puede leer desde un fichero utilizando la función read_corpus
stop_words=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
            'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
            'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are',
            'was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an',
            'the','and','but','if','or','because','as','until','while','of','at','by','for','with','about',
            'against','between','into','through','during','before','after','above','below','to','from','up',
            'down','in','out','on','off','over','under','again','further','then','once','here','there','when',
            'where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor',
            'not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'ever']


# función auxiliar. Se utiliza al obtener la representación mediante TF-IDF del texto pues en este caso
# se removerán las stop_words y se considerarán los "stem" en lugar de las palabrass
def english_stemmer(sentence):
    stemmer = EnglishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=stop_words, ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))


# guarda un pipeline entrenado
def save_model(model, modelName = "pickle_model.pkl"):
   pkl_filename = modelName
   with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)   


# carga un pipeline entrenado y guardado previamente
def load_model(rutaModelo = "pickle_model.pkl"):
  # Load from file
  with open(rutaModelo, 'rb') as file:
    pickle_model = pickle.load(file)
    return pickle_model 


# función auxiliar para realizar predicciones con el modelo
def predict_model01(model, data, pref='m'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  res = {}
  scores = None
  labels = model.predict(data)

  if hasattr(model, 'predict_proba'):
    scores = model.predict_proba(data)
  
    # empaquetar scores dentro de un diccionario que contiene labels, scores clase 1, scores clase 2, .... El nombre de la clase se normaliza a lowercase
    res = {f'scores_{pref}_{cls.lower()}':score for cls, score in zip(model.classes_, [col for col in scores.T])}

  # añadir datos relativos a la predicción
  res[f'labels_{pref}'] = labels

  # convertir a dataframe ordenando las columnas primero el label y luego los scores por clase, las clases ordenadas alfabéticamente.
  res = pd.DataFrame(res, columns=sorted(list(res.keys())))

  return res


# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model01(y_true, y_pred, y_score=None, pos_label='positive'):
  """
  
  """
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'y':0.95, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


print('Done!')

### Creación y entrenamiento del modelo 

En este apartado es donde se va a entrenar el modelo de clasificación.

En la siguiente porción de código se muestra el pipeline, se trata de un clasificador SVC, al que se le pasa la columna con el texto cientifico preprocesada.

Para el preproceso de la columna textual únicamente se aplica un CountVectorizer, para convertir el texto en una matriz de tokens, y posteriormente la técnica TF-IDF, para determinar la importancia de las palabras dentro del texto en comparación con los demas textos.

In [ ]:


def preprocess_pipeline():
    text_transformer = Pipeline([
        ('vect', CountVectorizer(analyzer=english_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True))
    ])

    preprocessor = ColumnTransformer([
        ('text1', text_transformer, "Comment")
    ], remainder='passthrough')

    classifier = SVC(probability=True)

    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

    return model

Seguidamente, al modelo creado se le ajsuta el conjunto de datos de entrenamiento.

In [ ]:


model01 = preprocess_pipeline()
model01.fit(train.loc[:, ["Comment"]], train['Topic'])

# Segundo modelo - Fine Tuning

El siguiente modelo trata de un reajuste de un modelo previamente entrenado, en este caso se trata de un modelo transoformers, en concreto el DistilBert. 

### Funciones necesarias

De igual manera, se definen las diferentes funciones necesarias para el entrenamiento del modelo

In [ ]:
# función auxiliar para obtener tensores de entrada al modelo a partir del texto
def get_model_inputs02(cfg, data):
  # obtener ids y máscaras para el conjunto de entrenamiento
  # no es necesario convertir a tensores porque la salida del tokenizador se encuentra en este formato,
  encodings = cfg['tokenizer'](data, truncation=True, padding='max_length', max_length=cfg['max_length'], return_tensors=cfg['framework'])


  # obtener representación tf-idf de cada instancia
  tfidf = cfg['vectorizer'].transform(data)
  tfidf_t = tf.convert_to_tensor(tfidf.toarray(), dtype='int32')

  # formatear los datos (tensores) de entrada de acuerdo con las opciones permitidas por TensorFlow
  # los nombres de las capas de Input creadas al construir el modelo ('input_ids', 'attention_mask', 'tfidf')
  # son utilizados como llaves en los diccionarios que representan las entradas al modelo
  inputs = {'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'tfidf': tfidf_t
           }

  return inputs


# función auxiliar para realizar predicciones con el modelo
def predict_model02(model, cfg, data, pref='m'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  res = {}
  inputs = get_model_inputs02(cfg, data)
  scores = model.predict(inputs)

  # empaquetar scores dentro de un diccionario que contiene labels, scores clase 1, scores clase 2, .... El nombre de la clase se normaliza a lowercase
  if cfg['num_labels']==1: # si es clasificación binaria, este modelo devuelve solo 1 score por instancia
    res = {f'scores_{pref}': scores[:,0]}
  else:
    res = {f'scores_{pref}_{cls.lower()}': score for cls, score in zip(cfg['label_binarizer'].classes_, [col for col in scores.T])}

  # añadir datos relativos a la predicción
  labels = cfg['label_binarizer'].inverse_transform(scores)
  res[f'labels_{pref}'] = labels

  # convertir a dataframe ordenando las columnas primero el label y luego los scores por clase, las clases ordenadas alfabéticamente
  res = pd.DataFrame(res, columns=sorted(list(res.keys())))
  return res


# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model02(y_true, y_pred, y_score=None, pos_label='positive'):
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'y':0.95, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()

def get_model_graph02(cfg):
  # cargar capa que representa al transformer, en este caso, TFDistilBertMainLayer
  transformer = TFDistilBertModel.from_pretrained(cfg['transformer_model_name'], return_dict=False).distilbert

  # crear los 'placeholder' correspondientes a las entradas del modelo
  # crear variable que representará las entradas de id para el Transformer
  input_ids = keras.layers.Input(shape=(cfg['max_length'],), name='input_ids', dtype='int32')

  # crear variable que representará las entradas de las máscaras para el Transformer
  input_masks = keras.layers.Input(shape=(cfg['max_length'],), name='attention_mask', dtype='int32')

  # crear variable que representará las entradas correspondientes a los rasgos específicos de dominio.
  input_tfidf = keras.layers.Input(shape=(cfg['number_of_additional_features'],), name='tfidf', dtype='float32')

  # indicar que TFDistilBertMainLayer se llama con input_ids e input_mask y capturar su salida, que contiene los embeddings correspondientes a cada token del texto
  # Existen varios criterios (ej. https://arxiv.org/pdf/1908.10084.pdf) sobre qué componenentes utilizar como rasgos,
  # en este caso, tomaremos el embedding correspondiente al token de inicio de texto [CLS] de modo similar a TFDistilBertForSequenceClassification
  transformer_output = transformer(input_ids, attention_mask=input_masks)

  # extraer embedding del token [CLS]
  # la transformación dependerá del tipo de salida del Transformer utilizado, en este caso TFDistilBertMainLayer
  # cuya salida es una tupla de un único elemento, que contiene un arreglo de dimensiones
  # (number_of_instances, number_of_tokens, embedding_dimension), donde el token 0 corresponde al CLS.
  transformes_cls_embedding = keras.layers.Lambda(lambda seq: seq[0][:,0,:], name='lambda')(transformer_output)

  # concatenar embedding del token [CLS] con el vector de rasgos adicionales.
  features = keras.layers.concatenate([transformes_cls_embedding, input_tfidf], name='concatenate')

  # establecer algunos hiper-parámetros del modelo
  initializer_range = 0.02
  hiden_units = 768
  seq_classif_dropout=0.2
  initializer = keras.initializers.TruncatedNormal(stddev=initializer_range)

  # crear pre_classifier, establecer como su entrada los rasgos concatenados (features).
  pre_classifier = keras.layers.Dense(hiden_units, kernel_initializer=initializer, activation='relu', name='pre_classifier')(features)

  # crear dropout layer y establecer como su entrada la salida de pre_classifier.
  dropout_layer = keras.layers.Dropout(rate=seq_classif_dropout, name='dropout')(pre_classifier)

  # crear classifier layer y establecer como su entrada la salida de la capa dropout.
  classifier = keras.layers.Dense(cfg['num_labels'], kernel_initializer=initializer, name='classifier')(dropout_layer)

  return input_ids, input_masks, input_tfidf, classifier

def configure_model02(input_ids, input_masks, input_tfidf, classifier):
  # definir algoritmo de optimización
  optimizer = keras.optimizers.Adam(learning_rate=5e-5)

  # definir función loss. Debe cuidarse que sea coherente con la salida esperada del modelo (vector de num_labels elementos)
  # y el formato de los ejemplos (vector one-hot de num_labels componentes para codificar las categorías)
  loss = keras.losses.BinaryCrossentropy(from_logits=True)

  # crear el modelo
  model = keras.Model(inputs=[input_ids, input_masks, input_tfidf], outputs=classifier, name='distilbert-custom') # conectar todos los nodos en un modelo

  # compilar el modelo, indicando otras métricas que se desee monitorear
  # La métrica debe ser apropiada para el tipo de problema (clasificación binaria o multiclase)
  #model.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
  model.compile(optimizer=optimizer, loss=loss, metrics=['categorical_accuracy'])

  return model

print('Done!')

### Creación y entrenamiento del modelo 

En la siguiente porción de código se muestran las configuraciones del modelo.

En este caso se han adaptado el código de ejemplo proporcionado por el profesorado y se ham modificado los valores de las siguientes configuraciones:
- num_labels
- number_of_additional_features

In [ ]:
cfg02 = {}  # diccionario para agrupar configuraciones y variables para su posterior uso
cfg02['framework'] = 'tf'  # TensorFlow como framework (por cuestiones del formato en los datos)
cfg02['max_length'] = 512  # máxima longitud de secuencia recomendada por DistilBERT
cfg02['transformer_model_name'] = 'distilbert-base-uncased'
cfg02['number_of_additional_features'] = 634  # específico al problema, en este caso, será la dimensión del vector tf-idf


cfg02['num_labels'] = 3  # cambiar este número según el número de clases

print('Done!')

Una vez instanciadas las configuraciones necesarias, se define el modelo y se muestran por pantalla.

In [ ]:
input_ids, input_masks, input_tfidf, classifier = get_model_graph02(cfg02)
model02 = configure_model02(input_ids, input_masks, input_tfidf, classifier)

# imprimir sumario del modelo
model02.summary()

# graficar el modelo (opcional)
model_image = tf.keras.utils.plot_model(model02, show_shapes=True, show_layer_names=True)
display(model_image)

print('Done!')

El siguiente paso es el de definir el tokenizador a utilizar, en este caso el proporcionado por el modelo transformer DistilBert, la vectorizacion TF-IDF, de la misma manera que para el primer modelo, y finalmente la función que convierte las etiquetas en vectores binarios.

In [ ]:
# cargar el tokenizador, disponible en Transformers
cfg02['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg02['transformer_model_name'] )

# instanciar TfidfVectorizer
cfg02['vectorizer'] = TfidfVectorizer(stop_words='english', max_features=cfg02['number_of_additional_features'])

# instanciar y entrenar LabelBinarizer
cfg02['label_binarizer'] = preprocessing.LabelBinarizer() # guardar para su posterior uso al decodificar predicciones

print('Done!')

A continuación se entrenan las funciones TfidfVectorizer y LabelBinarizer, para ajustarlas a los datos de entrenamiento, y finalmente en la celda siguiente se ajusta y entrena el modelo transformers.

In [ ]:
# entrenar TfidfVectorizer
cfg02['vectorizer'].fit(train[text_col].to_list())

# guardar TfidfVectorizer entrenado para su posterior uso (codificar nuevos datos).
with open('vectorizer_reviews.pkl', 'wb') as f:
    pickle.dump(cfg02['vectorizer'], f)

# entrenar LabelBinarizer
cfg02['label_binarizer'].fit(train[class_col])

# guardar LabelBinarizer para su uso posterior (decodificar las predicciones de nuevos datos)
with open('label_binarizer_reviews.pkl', 'wb') as f:
    pickle.dump(cfg02['label_binarizer'], f)

# obtener codificación one-hot
train_blabels = cfg02['label_binarizer'].transform(train[class_col])
val_blabes = cfg02['label_binarizer'].transform(val[class_col])

# obtener tensores correspondientes
train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')
val_blabels_t = tf.convert_to_tensor(val_blabes, dtype='int32')

# obtener diccionarios representando las entradas del modelo
train_inputs = get_model_inputs02(cfg02, train[text_col].to_list())
val_inputs = get_model_inputs02(cfg02, val[text_col].to_list())

print('Done!')

In [ ]:
# configuraciones
cfg02['checkpoints_dir'] = 'checkpoints'  # directorio donde se guardarán los checkpoints al entrenar el modelo
cfg02['model_name'] = 'distilbert-reviews'  # identificador al guardar los checkpoints
cfg02['trained_model_name'] = os.path.join(cfg02['checkpoints_dir'], cfg02['model_name'])

epochs_max = 10
epochs_to_save = 1 # si epochs_max % epochs_to_save !=0 podrían realizarse iteraciones extras
batch_size = 16

# ciclo de entrenamiento y guardar checkpoints
for epoch_current in range(0, epochs_max, epochs_to_save):
    epoch_from = epoch_current +1
    epoch_to = epoch_current + epochs_to_save
    print(f'Training model, epochs {epoch_from} - {epoch_to}')

    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model02.fit(train_inputs, y=train_blabels_t, initial_epoch=epoch_current, epochs=epoch_to, batch_size=batch_size, validation_data=(val_inputs,val_blabels_t))

    model02.save_weights(cfg02['trained_model_name'], save_format="tf")

print('Done!')

# Tercer modelo - Fine Tuning

Este último modelo entrenado es similar al anterior pero con ligeras modificaciones, como por ejemplo una capa dropout añadida para la regularización.

Ademas también se ha modificado el parámetro del número adicional de carácteristicas, que cambiará el tamaño de las capas dentro del modelo, por lo tanto también sus predicciones.

### Funciones necesarias

A continuación se muestran las funciones necesarias del modelo, en este caso a diferencia de los anteriores, se ha creado una nueva función de configuración para añadir el dropout mencionado anteriormente.

In [ ]:
# función auxiliar para obtener tensores de entrada al modelo a partir del texto
def get_model_inputs03(cfg, data):
  # obtener ids y máscaras para el conjunto de entrenamiento
  # no es necesario convertir a tensores porque la salida del tokenizador se encuentra en este formato,
  encodings = cfg['tokenizer'](data, truncation=True, padding='max_length', max_length=cfg['max_length'], return_tensors=cfg['framework'])


  # obtener representación tf-idf de cada instancia
  tfidf = cfg['vectorizer'].transform(data)
  tfidf_t = tf.convert_to_tensor(tfidf.toarray(), dtype='int32')

  # formatear los datos (tensores) de entrada de acuerdo con las opciones permitidas por TensorFlow
  # los nombres de las capas de Input creadas al construir el modelo ('input_ids', 'attention_mask', 'tfidf')
  # son utilizados como llaves en los diccionarios que representan las entradas al modelo
  inputs = {'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'tfidf': tfidf_t
           }

  return inputs


# función auxiliar para realizar predicciones con el modelo
def predict_model03(model, cfg, data, pref='m'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  res = {}
  inputs = get_model_inputs03(cfg, data)
  scores = model.predict(inputs)

  # empaquetar scores dentro de un diccionario que contiene labels, scores clase 1, scores clase 2, .... El nombre de la clase se normaliza a lowercase
  if cfg['num_labels']==1: # si es clasificación binaria, este modelo devuelve solo 1 score por instancia
    res = {f'scores_{pref}': scores[:,0]}
  else:
    res = {f'scores_{pref}_{cls.lower()}': score for cls, score in zip(cfg['label_binarizer'].classes_, [col for col in scores.T])}

  # añadir datos relativos a la predicción
  labels = cfg['label_binarizer'].inverse_transform(scores)
  res[f'labels_{pref}'] = labels

  # convertir a dataframe ordenando las columnas primero el label y luego los scores por clase, las clases ordenadas alfabéticamente
  res = pd.DataFrame(res, columns=sorted(list(res.keys())))
  return res


# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model03(y_true, y_pred, y_score=None, pos_label='positive'):
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'y':0.95, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()

def get_model_graph03(cfg):
  # cargar capa que representa al transformer, en este caso, TFDistilBertMainLayer
  transformer = TFDistilBertModel.from_pretrained(cfg['transformer_model_name'], return_dict=False).distilbert

  # crear los 'placeholder' correspondientes a las entradas del modelo
  # crear variable que representará las entradas de id para el Transformer
  input_ids = keras.layers.Input(shape=(cfg['max_length'],), name='input_ids', dtype='int32')

  # crear variable que representará las entradas de las máscaras para el Transformer
  input_masks = keras.layers.Input(shape=(cfg['max_length'],), name='attention_mask', dtype='int32')

  # crear variable que representará las entradas correspondientes a los rasgos específicos de dominio.
  input_tfidf = keras.layers.Input(shape=(cfg['number_of_additional_features'],), name='tfidf', dtype='float32')

  # indicar que TFDistilBertMainLayer se llama con input_ids e input_mask y capturar su salida, que contiene los embeddings correspondientes a cada token del texto
  # Existen varios criterios (ej. https://arxiv.org/pdf/1908.10084.pdf) sobre qué componenentes utilizar como rasgos,
  # en este caso, tomaremos el embedding correspondiente al token de inicio de texto [CLS] de modo similar a TFDistilBertForSequenceClassification
  transformer_output = transformer(input_ids, attention_mask=input_masks)

  # extraer embedding del token [CLS]
  # la transformación dependerá del tipo de salida del Transformer utilizado, en este caso TFDistilBertMainLayer
  # cuya salida es una tupla de un único elemento, que contiene un arreglo de dimensiones
  # (number_of_instances, number_of_tokens, embedding_dimension), donde el token 0 corresponde al CLS.
  transformes_cls_embedding = keras.layers.Lambda(lambda seq: seq[0][:,0,:], name='lambda')(transformer_output)

  # concatenar embedding del token [CLS] con el vector de rasgos adicionales.
  features = keras.layers.concatenate([transformes_cls_embedding, input_tfidf], name='concatenate')

  # establecer algunos hiper-parámetros del modelo
  initializer_range = 0.02
  hiden_units = 768
  seq_classif_dropout=0.2
  initializer = keras.initializers.TruncatedNormal(stddev=initializer_range)

  # crear pre_classifier, establecer como su entrada los rasgos concatenados (features).
  pre_classifier = keras.layers.Dense(hiden_units, kernel_initializer=initializer, activation='relu', name='pre_classifier')(features)

  # crear dropout layer y establecer como su entrada la salida de pre_classifier.
  dropout_layer = keras.layers.Dropout(rate=seq_classif_dropout, name='dropout')(pre_classifier)

  # crear classifier layer y establecer como su entrada la salida de la capa dropout.
  classifier = keras.layers.Dense(cfg['num_labels'], kernel_initializer=initializer, name='classifier')(dropout_layer)

  return input_ids, input_masks, input_tfidf, classifier
'''
def configure_model03(input_ids, input_masks, input_tfidf, classifier):
  # definir algoritmo de optimización
  optimizer = keras.optimizers.Adam(learning_rate=5e-5)

  # definir función loss. Debe cuidarse que sea coherente con la salida esperada del modelo (vector de num_labels elementos)
  # y el formato de los ejemplos (vector one-hot de num_labels componentes para codificar las categorías)
  loss = keras.losses.BinaryCrossentropy(from_logits=True)

  # crear el modelo
  model = keras.Model(inputs=[input_ids, input_masks, input_tfidf], outputs=classifier, name='distilbert-custom') # conectar todos los nodos en un modelo

  # compilar el modelo, indicando otras métricas que se desee monitorear
  # La métrica debe ser apropiada para el tipo de problema (clasificación binaria o multiclase)
  model.compile(optimizer=optimizer, loss=loss, metrics=['categorical_accuracy'])

  return model
'''

from tensorflow.keras.layers import Dropout

def configure_model03(input_ids, input_masks, input_tfidf, classifier):
    optimizer = keras.optimizers.Adam(learning_rate=5e-5)

    loss = keras.losses.BinaryCrossentropy(from_logits=True)

    model_input = [input_ids, input_masks, input_tfidf]
    x = classifier
    # Dropout del 30%
    x = Dropout(0.3)(x)
    outputs = keras.layers.Dense(cfg03['num_labels'], activation='softmax')(x)
    model = keras.Model(inputs=model_input, outputs=outputs, name='distilbert-custom')

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model

print('Done!')

### Creación y entrenamiento del modelo 

En este punto es donde aparece diferenciación con el modelo anterior, en este caso el valor del número de caracteristicas adicionales es 324, que como se verá al final, cambia el comportamiento y la arquitectura de parámetros.

In [ ]:
cfg03 = {}  # diccionario para agrupar configuraciones y variables para su posterior uso
cfg03['framework'] = 'tf'  # TensorFlow como framework (por cuestiones del formato en los datos)
cfg03['max_length'] = 512  # máxima longitud de secuencia recomendada por DistilBERT
cfg03['transformer_model_name'] = 'distilbert-base-uncased'
cfg03['number_of_additional_features'] = 324  # específico al problema, en este caso, será la dimensión del vector tf-idf


cfg03['num_labels'] = 3  # cambiar este número según el número de clases

print('Done!')

A continuación se grafica de manera visual el modelo de clasificación.

In [ ]:
input_ids, input_masks, input_tfidf, classifier = get_model_graph03(cfg03)
model03 = configure_model03(input_ids, input_masks, input_tfidf, classifier)

# imprimir sumario del modelo
model03.summary()

# graficar el modelo (opcional)
model_image = tf.keras.utils.plot_model(model03, show_shapes=True, show_layer_names=True)
display(model_image)

print('Done!')

En el gráfico anterior se puede ver el nuevo modelo creado, con unas pequeñas diferencias que el modelo 02.

A continuación se procede de manera identica, ajustando al modelo los datos de entrenamiento.

In [ ]:
# cargar el tokenizador, disponible en Transformers
cfg03['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg03['transformer_model_name'] )

# instanciar TfidfVectorizer
cfg03['vectorizer'] = TfidfVectorizer(stop_words='english', max_features=cfg03['number_of_additional_features'])

# instanciar y entrenar LabelBinarizer
cfg03['label_binarizer'] = preprocessing.LabelBinarizer() # guardar para su posterior uso al decodificar predicciones

print('Done!')

In [ ]:
# entrenar TfidfVectorizer
cfg03['vectorizer'].fit(train[text_col].to_list())

# guardar TfidfVectorizer entrenado para su posterior uso (codificar nuevos datos).
with open('vectorizer_reviews.pkl', 'wb') as f:
    pickle.dump(cfg03['vectorizer'], f)

# entrenar LabelBinarizer
cfg03['label_binarizer'].fit(train[class_col])

# guardar LabelBinarizer para su uso posterior (decodificar las predicciones de nuevos datos)
with open('label_binarizer_reviews.pkl', 'wb') as f:
    pickle.dump(cfg03['label_binarizer'], f)

# obtener codificación one-hot
train_blabels = cfg03['label_binarizer'].transform(train[class_col])
val_blabes = cfg03['label_binarizer'].transform(val[class_col])

# obtener tensores correspondientes
train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')
val_blabels_t = tf.convert_to_tensor(val_blabes, dtype='int32')

# obtener diccionarios representando las entradas del modelo
train_inputs = get_model_inputs03(cfg03, train[text_col].to_list())
val_inputs = get_model_inputs03(cfg03, val[text_col].to_list())

print('Done!')

In [ ]:
# configuraciones
cfg03['checkpoints_dir'] = 'checkpoints'  # directorio donde se guardarán los checkpoints al entrenar el modelo
cfg03['model_name'] = 'distilbert-reviews'  # identificador al guardar los checkpoints
cfg03['trained_model_name'] = os.path.join(cfg03['checkpoints_dir'], cfg03['model_name'])

epochs_max = 10
epochs_to_save = 1 # si epochs_max % epochs_to_save !=0 podrían realizarse iteraciones extras
batch_size = 16

# ciclo de entrenamiento y guardar checkpoints
for epoch_current in range(0, epochs_max, epochs_to_save):
    epoch_from = epoch_current +1
    epoch_to = epoch_current + epochs_to_save
    print(f'Training model, epochs {epoch_from} - {epoch_to}')

    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model03.fit(train_inputs, y=train_blabels_t, initial_epoch=epoch_current, epochs=epoch_to, batch_size=batch_size, validation_data=(val_inputs,val_blabels_t))

    model03.save_weights(cfg03['trained_model_name'], save_format="tf")

print('Done!')

# Construcción del Ensemble

En este punto ya se disponen de 3 modelos de clasificación textual entrenados, por lo que el siguiente y último paso es el de crear el ensemble de los modelos y entrenarlo mediante los datos de entrenamiento.

Se va a seguir la estrategia de construcción manual del ensemble, la empleada por el profesorado en las notebooks proporcionadas.

Por lo que el primer paso es el de hacer predicciones sobre cada uno de los 3 modelos, para asi obtener las probabilidades de pertenencia a cada una de las 3 clases.

In [ ]:
data = train

In [ ]:
m01_pred = predict_model01(model01, data[[text_col]], pref='m01')

print(f'\n {m01_pred.head(5)}')
print('Done!')

In [ ]:
m02_pred = predict_model02(model02, cfg02, data[text_col].to_list(), pref='m02')
print(f'\n {m02_pred.head(5)}')
print('Done!')

In [ ]:
m03_pred = predict_model03(model03, cfg03, data[text_col].to_list(), pref='m03')
print(f'\n {m03_pred.head(5)}')
print('Done!')

Una vez obtenidas todas las probabilidades, se van a concatenar en la matriz cmb, a la que posteriormente se eliminarán las columnas con las etiquetas para permanecer unicamente con las probabilidades obtenidas por cada modelo.

In [ ]:
cmb = pd.concat([m01_pred, m02_pred, m03_pred], axis=1)
cmb

In [ ]:
cmb.drop(columns=['labels_m01', 'labels_m02', 'labels_m03'], inplace=True)

X = cmb.values
y = data[class_col] # !ATENCIÓN! el orden de las instancias debe ser el mismo en X e y

print(cmb.head(5))
print('Done!')

En este punto ya disponemos en la variable X con las probabilidades de cada clase y con la variable y en la que se almacenan las categorias reales de los textos.

El siguiente paso es el de entrenar el clasificador con los datos generados, en este caso al igual que las notebooks proporcionadas en la asignatura, se ha entrenado un arbol de decision que dispone de la ventaja de interpretabilidad al dibujar el arbol.

In [ ]:
# instanciar el clasificador
classifier = DecisionTreeClassifier(criterion='entropy', random_state=seed)

# entrenar el clasificador
classifier.fit(X, y)

# graficar el árbol
fig = plt.figure(figsize=(20,20))

plot_tree(classifier, feature_names=cmb.columns, class_names=classifier.classes_, filled=True, impurity=False)
plt.show()

print('Done!')

El árbol generado permite interpretar el funcionamiento del ensemble, la ecuación úbicada en la parte superior de cada nodo permite analizar las diferentes divisiones hacia los nodos hijo y las diferentes clases que se predicen.

Como se observa al principio utiliza principalmente los scores del modelo 02, esto permite predecir mejor los textos que hablan sobre biologia, posteriormente en las ramas inferiores utiliza el modelo 03 para predecir textos quimicos y el 02 para predecir textos sobre fisica.

# Evaluacion del ensemble y de los 3 modelos

En este punto ya se disponen de los tres modelos entrenados y del ensemble de clasificación creado. Por lo que el siguiente paso es analizar y comparar sus resultados sobre el conjunto de validación.

In [ ]:
data = val

## Evaluación del primer modelo

In [ ]:
# predecir y evaluar conjunto de validación con el modelo 1
true_labels = data[class_col]

m01_pred = predict_model01(model01, data[["Comment"]], pref='m01')
if 'scores_m_positive' not in m01_pred:
    m01_pred['scores_m_positive'] = 0


# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
evaluate_model01(true_labels, m01_pred['labels_m01'], m01_pred['scores_m_positive'], 'positive')

print('Done!')

## Evaluación del segundo modelo

In [ ]:
true_labels = data[class_col]

m02_pred = predict_model02(model02, cfg02, data[text_col].to_list(), pref='m02')

evaluate_model02(true_labels, m02_pred['labels_m02'])  # notar que en este caso se no suministran los scores

print('Done!')

## Evaluación del tercer modelo

In [ ]:
true_labels = data[class_col]

m03_pred = predict_model03(model03, cfg03, data[text_col].to_list(), pref='m03')

evaluate_model03(true_labels, m03_pred['labels_m03'])  # notar que en este caso se no suministran los scores

print('Done!')

## Evaluacion del Ensemble

### Función necesaria

A continuación se muestra la función necesaria para la validación del ensemble de modelos.

In [ ]:
# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'y':0.95, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


### Evaluación

Para la evaluación del ensemble, es necesario proceder de la misma manera que para su creación y entrenamiento.

Por lo que a continuación se crea nuevamente la matriz X con las probabilidades calculadas sobre el conjunto de datos de validación y la matriz y que dispone de las etiquetas reales de los textos.

In [ ]:
# combinar predicciones de los clasificadores base
cmb = pd.concat([m01_pred, m02_pred, m03_pred], axis=1)
cmb.drop(columns=['labels_m01', 'labels_m02', 'labels_m03', 'scores_m_positive'], inplace=True)

X = cmb.values
y = data[class_col]  # !ATENCIÓN! el orden de las instancias debe ser el mismo en X e y

print(cmb.head(5))
print('Done!')

In [ ]:
# predecir y evaluar conjunto de validación con el modelo 3
true_labels = y

mc_pred = predict_model01(classifier, X, pref='mc')

evaluate_model(true_labels, mc_pred['labels_mc'])

print('Done!')

# Resultados y Conclusiones

En la siguiente tabla se muestran los resultados de Accuracy obtenidos por los tres modelos.

| Modelo | Accuracy |
|--------|----------|
| 01     | 65.60    |
| 02     | 69.33    |
| 03     | 66.93    |

Mientras que el ensemble obtiene un **70.93** de acierto en las predicciones, por lo que el ensemble creado utilizado los 3 modelos es mas preciso en cuanto a la predicción de la clase de los textos cientificos.

Al analizar las matrices de confusión tambien se observa las predicciones que realizan los modelos, por ejemplo, el modelo03 tiende a tener mejores predicciones sobre los textos quimicos, mientras que los demás modelos y el ensemble presentan unas predicciones mas similares en cada clase.

Como último aspecto interesante a comentar, al realizar diferentes ejecuciones con diferentes números de instancias para cada clase, se ha observado que a medida que el conjunto de datos crece, el porcentaje de acierto de cada modelo disminuye. Esto podria indicar que los modelos generados tienden al sobreentrenamiento ya que presentan malos resultados sobre el conjunto de validación.